# What is pwntools?



It is a set of security libraries allowing to:
* connect to a distant server;
* read and send information;
* manipulate bits, hex, strings easily;
* access command-line arguments easily;
* do many security-related complex things (interact with gdb, create shellcodes, etc.)

# Installation



## Installation on your own computer (or on an ISAE SUPAERO lab computer)

Start checking if you already have pwntools:

```
$ python3
Python 3.6.9 (default, Nov  7 2019, 10:44:02) 
[GCC 8.3.0] on linux
Type "help", "copyright", "credits" or "license" for more information.
>>> from pwn import *

```

if it does not raise an error you have it. If there is an error you can install it with:

```
$ sudo -H pip3 --proxy=https://proxy.isae.fr:3128 install 'pwntools>=4.0.0beta0'
```

removing the proxy option if you are not inside ISAE.




## Installation in this notebook

Pwntools is not installed by default in the computer executing this notebook. 

Execute the next cell to install it (no need of sudo or proxy in this notebook).


In [ ]:
!pip3 install 'pwntools>=4.0.0beta0'

     |████████████████████████████████| 11.7 MB 7.5 MB/s 
     |████████████████████████████████| 75 kB 5.1 MB/s 
     |████████████████████████████████| 2.1 MB 45.6 MB/s 
     |████████████████████████████████| 151 kB 79.2 MB/s 
     |████████████████████████████████| 90 kB 12.1 MB/s 
     |████████████████████████████████| 210 kB 77.1 MB/s 
     |████████████████████████████████| 8.3 MB 24.2 MB/s 
     |████████████████████████████████| 68 kB 8.1 MB/s 
     |████████████████████████████████| 61 kB 562 kB/s 
     |████████████████████████████████| 3.6 MB 46.0 MB/s 
     |████████████████████████████████| 856 kB 73.8 MB/s 
     |████████████████████████████████| 117 kB 70.0 MB/s 
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26119 sha256=25b952f04b4cd26608b0c674999e6a7b4fe84dfee5e1f36c187c92581fbff976
  Stored in directory: /root/.cache/pip/wheels/16/85/bd/1001cbb46dcfb71c2001cd7401c6fb250392f22a81ce3722f7
  Created wheel for colored-traceback:

Now we can import the pwntools functions without error. As pwntools is supposed to be executed in a terminal and not on a Jupyter notebook, we must first define an environment variable before importing the tools in pwntools.

In [ ]:
import os; os.environ['PWNLIB_NOTERM']="True"

In [ ]:
from pwn import *

Now we are ready. Let's go!

# Connecting to a remote and interacting

Here is a first example for `remote` that comes from the [socket documentation](https://docs.pwntools.com/en/stable/tubes/sockets.html)

In [ ]:
from pwn import *
r = remote('google.com', 443, ssl=True)
# HTML hint for next line.
# The \r\n\r\n is just what HTML requires to understand that you have ended your
# command and not doing a newline. So basically you ask "GET /" and say "over".
r.send("GET /\r\n\r\n")
reply = r.readline()
print(f"Server reply: {reply}")
r.close()

[x] Opening connection to google.com on port 443
[x] Opening connection to google.com on port 443: Trying 74.125.20.139
[+] Opening connection to google.com on port 443: Done
Server reply: b'HTTP/1.0 200 OK\r\n'
[*] Closed connection to google.com port 443


The remote function opens a TCP connexion with a server (first argument) on a given port (second argument).

Some important remarks:
* The `ssl=True` option will add to the TCP negociation (which opens the connection) an SSL negociation (which secures the TCP connection), before the message exchanges start. This happens in the previous example as the Google webpage is accessible in HTTPS only (which is just HTTP messages inside an SSL-secured connectoin).
* To interact with a remote the `sendline` and `readline` functions are used in general. The input of `sendline` is a bytearray to which a newline is added before being sent to the remote. The function `readline` retrieves a bytearray up to the first newline character and returns it (without removing this newline character).
* HTTP being punctilious with newlines and carriage returns, the example uses the function `send` with sends the input bytearray as is.
* For `send` and for `sendline` if you use a string as input, the function will encode it as a bytearray before sending it.




## Beware of the bytearray/string conversions

Let's expand a little bit the example above to focus a bit on the reply format.

In [ ]:
from pwn import *
r = remote('google.com', 443, ssl=True)
r.send("GET /\r\n\r\n")
reply = r.readline()
print(f"Server reply: {reply}")
print(f"Server reply as a string: {reply.decode()}")
r.close()
if str(reply).startswith("HTTP"): # Bad idea needs decoding
  # We never get here
  print("We are in the first if")
if reply.decode().startswith("HTTP"): # This is the right way to do it
  print("We are in the second if")
r.close()
  

[x] Opening connection to google.com on port 443
[x] Opening connection to google.com on port 443: Trying 74.125.135.139
[+] Opening connection to google.com on port 443: Done
Server reply: b'HTTP/1.0 200 OK\r\n'
Server reply as a string: HTTP/1.0 200 OK

[*] Closed connection to google.com port 443
We are in the second if


This example shows that:
  * The received bytearray can be decoded to a string using the `decode()` function to obtain a string
  * We can use on a decoded bytearray, usual (and useful) string functions such as `startswith()`

**Note:** You should not try to build strings from bytearrays in any other way than with `decode()` (nor transform strings to bytearray with anything else than `encode()`). If you do, you will run into problems.



## Beware of newlines

Le's show now a small issue. Note that in the following code

```
# Suppose we have a remote r
myline=r.readline()
r.sendline(myline)
```

we are not sending what we received. This is simply because `readline` keeps the final newline in the line read and `sendline` adds a second newline after it. So if `myline == b'Hello\n'` then we are sending `b'Hello\n\n'`.

It is possible to tell `readline` not to keep the final newline with `readline(keepends=False)`. Another option is to strip the received byterray with `r.readline().strip()`. Be careful though, as `strip()` strips all whitespace at the beginning or end ot the bytearray.

Finally note that there is a `readlines(number)` function that returns a list of `number` lines. This functions does NOT keep newlines at the end of each bytearray. This change of behaviour can sometimes lead to errors.

# Command line arguments



Pwntools gives you easy access to command-line defined uppercase variables. Note that in a jupyter notebook there is no "command-line arguments" for a given cell. We thus first create a local file and will call it with magic commands.

In [ ]:
with open("/tmp/mypythonscript.py", 'wt') as f:
  f.write(r"""from pwn import *
# Call the script with:
# ./mypythonscript.py REMOTE=agivenremote PORT=agivenport

# We can allow script calls with default values
if not args.REMOTE: args.REMOTE='1.1.1.1'
if not args.PORT: args.PORT=80

try:
  rem = remote(args.REMOTE, args.PORT)
except:
  print('Remote connection failed!')

rem.send("GET /\r\n\r\n".encode())
print(f'Server reply: {rem.recvall()}')
rem.close()
""")
  f.close()

Let's now execute the script, with or without command-line arguments.

In [ ]:
!python3 /tmp/mypythonscript.py


[x] Opening connection to 1.1.1.1 on port 80
[x] Opening connection to 1.1.1.1 on port 80: Trying 1.1.1.1
[+] Opening connection to 1.1.1.1 on port 80: Done
[x] Receiving all data
[x] Receiving all data: 0B
[x] Receiving all data: 155B
[+] Receiving all data: Done (155B)
[*] Closed connection to 1.1.1.1 port 80
Server reply: b'<html>\r\n<head><title>400 Bad Request</title></head>\r\n<body>\r\n<center><h1>400 Bad Request</h1></center>\r\n<hr><center>cloudflare</center>\r\n</body>\r\n</html>\r\n'


In [ ]:
!python3 /tmp/mypythonscript.py REMOTE="www.google.fr"


[x] Opening connection to www.google.fr on port 80
[x] Opening connection to www.google.fr on port 80: Trying 74.125.20.94
[+] Opening connection to www.google.fr on port 80: Done
[x] Receiving all data
[x] Receiving all data: 0B
[x] Receiving all data: 4.00KB
[x] Receiving all data: 8.00KB
[x] Receiving all data: 12.00KB
[x] Receiving all data: 16.00KB
[x] Receiving all data: 20.00KB
[x] Receiving all data: 24.00KB
[x] Receiving all data: 28.00KB
[x] Receiving all data: 32.00KB
[x] Receiving all data: 36.00KB
[x] Receiving all data: 40.00KB
[x] Receiving all data: 44.00KB
[x] Receiving all data: 48.00KB
[x] Receiving all data: 48.42KB
[+] Receiving all data: Done (48.42KB)
[*] Closed connection to www.google.fr port 80
Server reply: b'HTTP/1.0 200 OK\r\nDate: Fri, 11 Dec 2020 08:07:57 GMT\r\nExpires: -1\r\nCache-Control: private, max-age=0\r\nContent-Type: text/html; charset=ISO-8859-1\r\nP3P: CP="This is not a P3P policy! See g.co/p3phelp for more info."\r\nServer: gws\r\nX-XSS-Prote

Note that we have used a new function, `recvall`, to receive data that allows to get all data until an EOF is sent by the remote.

# Parse module



## Installation



Before using it we first need to install the parse module.

In [ ]:
!pip3 install parse

  Created wheel for parse: filename=parse-1.18.0-cp36-none-any.whl size=24133 sha256=bedae6cc7d469eba1363b339bd275bf298ec83f4432a2dfaf764bed187693c07
  Stored in directory: /root/.cache/pip/wheels/2a/53/09/869ca5781ede342254ffac09ca99461b008c3e5f8dd079b0c0
Successfully built parse


## One variable parsing



Now let's dive into an example

In [ ]:
from pwn import *
import parse
rem = remote("1.1.1.1", 80)
# Get the full reply
rem.send("GET /\r\n\r\n")
reply = rem.recvall()
print(f"Full reply: {reply}")
reply_as_a_string = reply.strip().decode()
# When we perfectly know the reply format we can extract part of the reply easily
parsed = parse.parse('<html>{}</html>', reply_as_a_string)
if not parsed:
  print("The reply is not html code")
else:
  print(f"The code inside the html tags is {parsed[0].strip().encode()}")
rem.close()

[x] Opening connection to 1.1.1.1 on port 80
[x] Opening connection to 1.1.1.1 on port 80: Trying 1.1.1.1
[+] Opening connection to 1.1.1.1 on port 80: Done
[x] Receiving all data
[x] Receiving all data: 0B
[x] Receiving all data: 155B
[+] Receiving all data: Done (155B)
[*] Closed connection to 1.1.1.1 port 80
Full reply: b'<html>\r\n<head><title>400 Bad Request</title></head>\r\n<body>\r\n<center><h1>400 Bad Request</h1></center>\r\n<hr><center>cloudflare</center>\r\n</body>\r\n</html>\r\n'
The code inside the html tags is b'<head><title>400 Bad Request</title></head>\r\n<body>\r\n<center><h1>400 Bad Request</h1></center>\r\n<hr><center>cloudflare</center>\r\n</body>'


Some comments on this example:
* Note that, when parsing, the string `'<html>{}</html>'` must match entirely the string `reply_as_a_string`. In particular if there is something before `<html>` or after `</html>` in `reply_as_a_string` (e.g. a newline) it will not match.
* We define `reply_as_a_string` by first calling `strip()` to remove newlines that would prevent the format used in the parse function to match the result.
* We then call `decode()` to get a string out of the stripped bytearray.
* When the parsing fails no exception is lifted but the return value will be `None`, the check `if not parsed:` is important after each parsing to detect such events.
* The parsing results are in a list, as here we only have one value parsed we
get it with `parsed[0]`. There can be more than a value parsed as it will be shown in the next example.
* In order to remove useless carriage return and newlines around the title and to print a single-line bytearray instead of a multiline string we print `parsed[0].strip().encode()` instead of just `parsed[0]`



## Multiple variables parsing

Here is a slightly more evolved example in which we grab two variables

In [ ]:
from pwn import *
import parse
rem = remote("1.1.1.1", 80)
# Get the full reply
rem.send("GET /\r\n\r\n")
reply = rem.recvall()
print(f"Full reply: {reply}")
reply_as_a_string = reply.strip().decode()
# We can parse multiple variables
parsed = parse.parse(b'<html>\r\n<head>{}</head>\r\n<body>\r\n{}</body>\r\n</html>'.decode(), reply_as_a_string)
if not parsed:
  print("The reply is not formated as expected")
else:
  print(f"The code inside the head tags is {parsed[0].strip().encode()}")
  print(f"The code inside the body tags is {parsed[1].strip().encode()}")
rem.close()

[x] Opening connection to 1.1.1.1 on port 80
[x] Opening connection to 1.1.1.1 on port 80: Trying 1.1.1.1
[+] Opening connection to 1.1.1.1 on port 80: Done
[x] Receiving all data
[x] Receiving all data: 0B
[x] Receiving all data: 155B
[+] Receiving all data: Done (155B)
[*] Closed connection to 1.1.1.1 port 80
Full reply: b'<html>\r\n<head><title>400 Bad Request</title></head>\r\n<body>\r\n<center><h1>400 Bad Request</h1></center>\r\n<hr><center>cloudflare</center>\r\n</body>\r\n</html>\r\n'
The code inside the head tags is b'<title>400 Bad Request</title>'
The code inside the body tags is b'<center><h1>400 Bad Request</h1></center>\r\n<hr><center>cloudflare</center>'


Note that in the parse command we used a bytearray to define newlines as HTML sends them (in general newlines are just '\n') and then converted it to a string with `decode()`.

## Wildcards

Unfortunately there is no wildcards/regular expressions in the parse module. Something like `parse.parse('.*<head>{}</head>.*', mystring)` will not work. We can circumvent this problem by adding parsed variables that won't be used.

In [ ]:
from pwn import *
import parse
rem = remote("1.1.1.1", 80)
# Get the full reply
rem.send("GET /\r\n\r\n")
reply = rem.recvall()
print(f"Full reply: {reply}")
reply_as_a_string = reply.strip().decode()
# We can parse the body without knowing the exact format up to it
parsed = parse.parse('{}<body>{}</body>{}', reply_as_a_string)
if not parsed:
  print("The reply does not have a body")
else:
  print(f"The code inside the body tags is: {parsed[1].strip().encode()}")
  #We have just ignored what was before (parsed[0]) and after (parsed[2]) the body
rem.close()

[x] Opening connection to 1.1.1.1 on port 80
[x] Opening connection to 1.1.1.1 on port 80: Trying 1.1.1.1
[+] Opening connection to 1.1.1.1 on port 80: Done
[x] Receiving all data
[x] Receiving all data: 0B
[x] Receiving all data: 155B
[+] Receiving all data: Done (155B)
[*] Closed connection to 1.1.1.1 port 80
Full reply: b'<html>\r\n<head><title>400 Bad Request</title></head>\r\n<body>\r\n<center><h1>400 Bad Request</h1></center>\r\n<hr><center>cloudflare</center>\r\n</body>\r\n</html>\r\n'
The code inside the body tags is: b'<center><h1>400 Bad Request</h1></center>\r\n<hr><center>cloudflare</center>'


# Your first challenge

Get connected to the ctfd server (follow the instructions in the LMS or ask the teacher for the IP), click on register (top-right corner), use your real name and ISAE email to create an account, and log in. You will have a set of challenges. The remotehost to do the challenges on is : hidden (again follow the LMS or teacher instructions to get the IP).  

In [ ]:
# Put your code here